# Chapter 11-01: GPU 아키텍처 기초 (GPU Architecture Basics)

## 학습 목표
- GPU의 **SM(Streaming Multiprocessor)**, **Warp**, **Thread Block** 개념과 이들의 계층적 관계를 정확히 이해한다
- GPU 메모리 계층 (**Global / Shared / Register**)의 용량, 속도, 접근 방식을 CPU와 비교한다
- **Arithmetic Intensity** 지표로 연산이 Compute-bound인지 Memory-bound인지 판별하는 Roofline 분석을 수행한다
- 딥러닝의 핵심 연산(GEMM, Softmax, LayerNorm)의 AI를 계산하고, GPU에서 최적화 방향을 제시한다

## 목차
1. [수학적 기초: Arithmetic Intensity와 Roofline Model](#1.-수학적-기초)
2. [GPU vs CPU 아키텍처 비교](#2.-GPU-vs-CPU)
3. [메모리 계층 구조 심화](#3.-메모리-계층)
4. [Roofline 분석: 딥러닝 연산 분류](#4.-Roofline-분석)
5. [Warp과 Occupancy 이해](#5.-Warp과-Occupancy)
6. [정리 및 연습 문제](#6.-정리)

## 1. 수학적 기초

### Arithmetic Intensity (산술 집약도)

$$\text{AI} = \frac{\text{FLOPs}}{\text{메모리 접근량 (Bytes)}}$$

### Roofline Model: 실제 성능 상한

$$\text{Attainable Performance} = \min\left(\text{Peak FLOPs}, \; \text{AI} \times \text{Peak BW}\right)$$

- **Memory-bound**: $\text{AI} < \text{Operational Intensity}_{ridge}$ → 메모리 대역폭이 병목
- **Compute-bound**: $\text{AI} > \text{Operational Intensity}_{ridge}$ → 연산 능력이 병목

Ridge Point (전환점):
$$\text{AI}_{ridge} = \frac{\text{Peak FLOPs}}{\text{Peak Memory BW}}$$

H100 예: $\frac{989 \text{ TFLOPS}}{3.35 \text{ TB/s}} \approx 295$ FLOPs/Byte

### 핵심 연산의 Arithmetic Intensity

| 연산 | FLOPs | 메모리 읽기 | AI | 구분 |
|------|-------|-----------|-----|------|
| GEMM ($m\times k \times n$) | $2mkn$ | $\approx 2(mk+kn+mn)$ bytes | $\sim mn/({m+n})$ | **Compute-bound** (큰 행렬) |
| Softmax ($S$ 원소) | $5S$ | $4S$ bytes (읽기+쓰기) | $\sim 1.25$ | **Memory-bound** |
| LayerNorm ($H$ 히든) | $7H$ | $8H$ bytes | $\sim 0.9$ | **Memory-bound** |
| Elementwise (add, relu) | $1S$ | $3S$ bytes | $\sim 0.33$ | **극단적 Memory-bound** |

---

### 🐣 초등학생을 위한 친절 설명!

#### 🏎️ GPU가 왜 AI 학습에 좋은가요?

> - **CPU**: 요리사 4명이 각자 복잡한 요리를 만드는 것 (소수의 강력한 코어)
> - **GPU**: 요리사 1만 명이 간단한 작업을 동시에 하는 것 (수천 개의 작은 코어)
>
> 딥러닝의 행렬 곱은 서로 독립적인 수십억 개의 작은 곱셈의 합 → GPU가 완벽!

#### 📊 Arithmetic Intensity가 왜 중요한가요?

> 요리사(GPU 코어)들이 재료(데이터)를 계속 기다리면 낭비예요!
> - **AI가 낮음**: 재료(메모리)를 가져오는 게 느려서 요리사들이 대기 → Memory-bound
> - **AI가 높음**: 재료는 충분, 요리사들이 풀가동 → Compute-bound (이상적!)

### 📝 연습 문제

#### 문제 1: GEMM의 Arithmetic Intensity

Llama-2 7B의 한 FFN 레이어: $A \in \mathbb{R}^{4096 \times 11008}$, 배치/시퀀스 합쳐 $X \in \mathbb{R}^{2048 \times 4096}$.  
이 GEMM의 FLOPs와 AI를 계산하고, H100 기준 Compute/Memory bound를 판별하라.

<details>
<summary>💡 풀이 확인</summary>

$m=2048, k=4096, n=11008$  
FLOPs = $2 \times 2048 \times 4096 \times 11008 \approx 184.5$ GFLOPS  
메모리 = $(2048\times4096 + 4096\times11008 + 2048\times11008) \times 2$ bytes ≈ 263 MB  
AI = $184.5\times10^9 / 263\times10^6 \approx \mathbf{701}$ FLOPs/Byte

H100 Ridge Point ≈ 295 FLOPs/Byte  
**AI (701) > Ridge (295) → Compute-bound** ✅ (GPU가 최대 활용됨)
</details>

#### 문제 2: Warp 오큐판시

H100 SM당 최대 2048 Thread, Warp 크기 32.  
커널이 Thread Block당 128 Thread, SM당 최대 Block 수 = 8이라면,  
이론 Occupancy(점유율)는?

<details>
<summary>💡 풀이 확인</summary>

SM당 활성 Thread = $128 \times 8 = 1024$  
SM당 활성 Warp = $1024 / 32 = 32$  
최대 Warp = $2048 / 32 = 64$  
Occupancy = $32/64 = \mathbf{50\%}$

낮은 Occupancy → 메모리 지연 숨기기 어려움 → 레지스터 사용량 줄여 Block 크기 늘릴 것
</details>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

## 2. GPU vs CPU 아키텍처 비교

In [ ]:
# ---------------------------------------------------
# GPU(H100) vs CPU(Intel Xeon) 스펙 비교
# ---------------------------------------------------

specs = {
    '항목': ['코어 수', 'Clock (GHz)', 'FP32 TFLOPS', '메모리 대역폭 (GB/s)', 'L1/SRAM (MB)', '전력 (W)'],
    'CPU\n(Intel Xeon 8490H)': ['60 코어', '1.9 (Boost 3.5)', '~3', '~300', '~240', '350W'],
    'GPU\n(NVIDIA H100 SXM)': ['16,896 CUDA 코어\n+528 TCs', '1.98 (Boost 2.28)', '989 (BF16: 1979)', '3,350', '~50 (Shared×132)', '700W'],
}

fig, ax = plt.subplots(figsize=(14, 4))
ax.axis('off')
table = ax.table(
    cellText=[[s, c, g] for s, c, g in zip(specs['항목'],
                                            specs['CPU\n(Intel Xeon 8490H)'],
                                            specs['GPU\n(NVIDIA H100 SXM)'])],
    colLabels=['항목', 'CPU (Intel Xeon 8490H)', 'GPU (NVIDIA H100 SXM)'],
    cellLoc='center', loc='center',
    colWidths=[0.25, 0.35, 0.4]
)
table.auto_set_font_size(False)
table.set_fontsize(10)
for (r, c), cell in table.get_celld().items():
    if r == 0:
        cell.set_facecolor('#1E88E5'); cell.set_text_props(color='white', fontweight='bold')
    elif c == 2:
        cell.set_facecolor('#E3F2FD')
    cell.set_edgecolor('white')
    cell.set_height(0.18)
ax.set_title('GPU vs CPU 아키텍처 비교 (H100 SXM vs Intel Xeon)', fontweight='bold', fontsize=12, pad=20)
plt.tight_layout()
plt.show()

print("핵심 차이:")
print(f"  FP32 성능: H100({989:.0f} TFLOPS) / Xeon({3:.0f} TFLOPS) = {989/3:.0f}배")
print(f"  메모리 BW: H100({3350:.0f} GB/s) / Xeon({300:.0f} GB/s) = {3350/300:.0f}배")
print(f"  코어 수:  H100(16,896) / Xeon(60) = {16896//60}배")

## 3. 메모리 계층

In [ ]:
# ---------------------------------------------------
# GPU 메모리 계층 구조 시각화
# ---------------------------------------------------

memory_hierarchy = [
    # (이름,  크기,          대역폭,      지연,    범위,       색)
    ('Register\n(스레드 전용)', '256KB/SM', '>100 TB/s', '~1 cycle', '스레드', '#1B5E20'),
    ('Shared Memory\n(SRAM)', '228KB/SM', '~10 TB/s',  '~5 cycles', 'Thread Block', '#1565C0'),
    ('L2 Cache',             '50 MB',    '~4 TB/s',   '~200 cycles', '전체 GPU', '#4527A0'),
    ('Global Memory\n(HBM)', '80 GB',    '3.35 TB/s', '~600 cycles', '전체 GPU', '#B71C1C'),
    ('CPU RAM (PCIe)',        '수 TB',    '~64 GB/s',  '~수천 cycles', '호스트', '#E65100'),
]

fig, ax = plt.subplots(figsize=(13, 7))
ax.set_xlim(0, 10); ax.set_ylim(-0.5, len(memory_hierarchy) + 0.5)
ax.axis('off')
ax.set_title('GPU 메모리 계층 구조 (H100 기준)\'\n속도↑ 용량↓ ← Register | HBM → 속도↓ 용량↑',
             fontsize=11, fontweight='bold')

widths = [9.5, 8.0, 6.5, 5.0, 3.5]

for i, ((name, size, bw, lat, scope, color), w) in enumerate(zip(reversed(memory_hierarchy), widths)):
    idx = len(memory_hierarchy) - 1 - i
    x_off = (10 - w) / 2
    rect = plt.Rectangle((x_off, idx * 0.95 + 0.05), w, 0.82,
                          color=color, alpha=0.80, ec='white', lw=2)
    ax.add_patch(rect)
    ax.text(5, idx * 0.95 + 0.46,
            f'{name}   |   {size}   |   BW: {bw}   |   지연: {lat}   |   범위: {scope}',
            ha='center', va='center', fontsize=9, color='white', fontweight='bold')

ax.annotate('', xy=(0.2, 4.7), xytext=(0.2, 0.2),
            arrowprops=dict(arrowstyle='<->', color='gray', lw=2))
ax.text(0.05, 2.5, '속도↑\n용량↓', ha='center', fontsize=9, color='gray', rotation=90)

plt.tight_layout()
plt.show()

print("메모리 설계 원칙:")
print("  ✅ 자주 쓰는 데이터 → Register / Shared Memory에 캐시")
print("  ✅ Global Memory(HBM) 접근 횟수 최소화 → Kernel Fusion의 핵심!")
print("  ✅ Coalesced Access: 인접한 스레드가 인접한 메모리 읽기 → 대역폭 최대 활용")

## 4. Roofline 분석

In [ ]:
# ---------------------------------------------------
# Roofline Model + 딥러닝 연산 분류
# ---------------------------------------------------

# H100 SXM 스펙
peak_compute_TFLOPS = 989.0    # FP32
peak_bw_TBps        = 3.35     # HBM
ridge_AI = peak_compute_TFLOPS / peak_bw_TBps   # FLOPs/Byte

print(f"H100 Ridge Point: {ridge_AI:.0f} FLOPs/Byte")
print(f"(AI < {ridge_AI:.0f}: Memory-bound, AI > {ridge_AI:.0f}: Compute-bound)\n")

# 딥러닝 연산별 AI 계산
ops = {
    # (이름, FLOPs/element, Bytes/element)
    'GEMM\n(m=4096,k=4096,n=4096)': (2 * 4096**3, (3 * 4096**2) * 2),  # 3 행렬 × FP16
    'GEMM\n(m=64,k=4096,n=4096)':   (2 * 64 * 4096 * 4096, (64*4096 + 4096**2 + 64*4096) * 2),
    'Self-Attention\n(S=2048,H=64)': (4 * 2048**2 * 64, 4 * 2048**2 * 2),
    'Softmax\n(S=2048)':             (5 * 2048, 4 * 2048 * 2),
    'LayerNorm\n(H=4096)':           (7 * 4096, 8 * 4096 * 2),
    'ReLU / GeLU\n(element-wise)':   (5, 3 * 2),  # per element
    'Dropout':                        (1, 3 * 2),
}

ai_values = {name: flops / bytes_ for name, (flops, bytes_) in ops.items()}

print(f"{'연산':<35} {'AI (FLOPs/Byte)':>18} {'구분'}")
print("-" * 65)
for name, ai in ai_values.items():
    bound = 'Compute-bound ⚡' if ai > ridge_AI else 'Memory-bound 🐌'
    clean_name = name.replace('\n', ' ')
    print(f"{clean_name:<35} {ai:>18.1f} {bound}")

# Roofline 시각화
ai_range = np.logspace(-1, 4, 500)
perf_limit = np.minimum(peak_compute_TFLOPS, ai_range * peak_bw_TBps)

fig, ax = plt.subplots(figsize=(11, 6))
ax.loglog(ai_range, perf_limit, 'k-', lw=3, label='Roofline (H100 SXM)')
ax.axvline(x=ridge_AI, color='gray', ls=':', lw=1.5, label=f'Ridge: {ridge_AI:.0f} FLOPs/B')

colors_ops = ['#1E88E5','#1E88E5','#43A047','#FB8C00','#FB8C00','#E53935','#E53935']
markers_ops = ['o', 's', 'D', '^', 'v', 'P', '*']

for (name, ai), c, m in zip(ai_values.items(), colors_ops, markers_ops):
    perf = min(peak_compute_TFLOPS, ai * peak_bw_TBps)
    ax.scatter(ai, perf, s=120, color=c, marker=m, zorder=5)
    ax.annotate(name.replace('\n', '\n'), (ai, perf),
                xytext=(ai * 1.2, perf * 0.8), fontsize=7.5,
                arrowprops=dict(arrowstyle='->', lw=0.6, color='gray'))

ax.fill_between(ai_range[ai_range < ridge_AI], perf_limit[ai_range < ridge_AI],
                alpha=0.06, color='red', label='Memory-bound 구간')
ax.fill_between(ai_range[ai_range > ridge_AI], perf_limit[ai_range > ridge_AI],
                alpha=0.06, color='blue', label='Compute-bound 구간')

ax.set_xlabel('Arithmetic Intensity (FLOPs / Byte)', fontsize=11)
ax.set_ylabel('Attainable Performance (TFLOPS)', fontsize=11)
ax.set_title('Roofline Model: H100 SXM + 딥러닝 연산 분류', fontweight='bold')
ax.legend(fontsize=9); ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n결론:")
print("  GEMM(large): Compute-bound → GPU 최대 활용 가능")
print("  Softmax, LayerNorm, Elementwise: Memory-bound → Kernel Fusion으로 HBM 접근 최소화 필요")

## 5. Warp과 Occupancy

In [ ]:
# ---------------------------------------------------
# Warp 개념 및 Occupancy 계산기
# ---------------------------------------------------

# H100 SM 스펙
h100_sm_specs = {
    'SM 수':              132,
    '최대 Thread/SM':    2048,
    '최대 Block/SM':        32,
    'Warp 크기':            32,
    'Register/SM (K)':      256,   # 256K 32-bit registers
    'Shared Memory/SM (KB)': 228,
}

print("=== H100 SM (Streaming Multiprocessor) 스펙 ===")
for k, v in h100_sm_specs.items():
    print(f"  {k:<28}: {v}")

print("\n=== Occupancy 계산기 ===")
print(f"{'Block 크기':>12} | {'Block/SM':>8} | {'Thread/SM':>10} | {'Warp/SM':>8} | {'Occupancy':>10}")
print("-" * 58)

max_threads = h100_sm_specs['최대 Thread/SM']
max_blocks  = h100_sm_specs['최대 Block/SM']
warp_size   = h100_sm_specs['Warp 크기']
max_warps   = max_threads // warp_size

for block_size in [32, 64, 128, 256, 512, 1024]:
    blocks_by_threads = max_threads // block_size
    active_blocks = min(blocks_by_threads, max_blocks)
    active_threads = active_blocks * block_size
    active_warps   = active_threads // warp_size
    occupancy      = active_warps / max_warps
    print(f"{block_size:>12} | {active_blocks:>8} | {active_threads:>10} | {active_warps:>8} | {occupancy:>9.1%}")

print(f"\nH100 최대 Warp/SM: {max_warps}")
print("Block 크기 256이 100% Occupancy에서 권장 (실제 레지스터 사용량에 따라 다름)")

In [ ]:
# ---------------------------------------------------
# GPU 스레드 계층 시각화
# ---------------------------------------------------

import matplotlib.patches as mpatches

fig, ax = plt.subplots(figsize=(13, 5))
ax.set_xlim(0, 13); ax.set_ylim(0, 6)
ax.axis('off')
ax.set_title('GPU 스레드 계층: Grid → Block → Warp → Thread', fontsize=12, fontweight='bold')

# Grid
grid = plt.Rectangle((0.3, 0.5), 12.4, 5, color='#E8EAF6', ec='#3949AB', lw=2.5, zorder=1)
ax.add_patch(grid)
ax.text(6.5, 5.3, 'Grid (커널 전체 실행 단위)', ha='center', fontsize=10, fontweight='bold', color='#3949AB')

# Thread Blocks
block_positions = [(0.6, 2.8), (4.7, 2.8), (8.8, 2.8)]
for bx, by in block_positions:
    block = plt.Rectangle((bx, by), 3.8, 2.3, color='#E3F2FD', ec='#1565C0', lw=2, zorder=2)
    ax.add_patch(block)
    ax.text(bx+1.9, by+2.05, 'Thread Block (128 threads)', ha='center', fontsize=8, color='#1565C0', fontweight='bold')

    # Warps within each block
    for w in range(4):
        wx = bx + 0.1 + w * 0.9
        wy = by + 0.2
        warp = plt.Rectangle((wx, wy), 0.8, 1.6, color='#BBDEFB', ec='white', lw=1, zorder=3)
        ax.add_patch(warp)
        ax.text(wx+0.4, wy+1.3, f'W{w}', ha='center', fontsize=8, fontweight='bold', color='#1565C0')
        # Threads in warp
        for t in range(4):
            tx = wx + 0.15 + (t % 2) * 0.3
            ty = wy + 0.15 + (t // 2) * 0.4
            ax.scatter(tx, ty, s=30, color='#1565C0', zorder=4)

# SM 배치도
ax.text(6.5, 2.4, '↑ 위: Thread Hierarchy', ha='center', fontsize=9, color='gray')

# SM → 하단 메모리 계층
for i, (mem, y, color) in enumerate([
    ('Register (스레드 전용)', 1.9, '#1B5E20'),
    ('Shared Memory (Block 공유)', 1.4, '#1565C0'),
    ('Global Memory/HBM (전체)', 0.9, '#B71C1C'),
]):
    rect = plt.Rectangle((0.4, y), 12.2, 0.38, color=color, alpha=0.7, ec='white', lw=1, zorder=2)
    ax.add_patch(rect)
    ax.text(6.5, y + 0.19, mem, ha='center', va='center', fontsize=9, color='white', fontweight='bold')

plt.tight_layout()
plt.show()

## 6. 정리

### 핵심 개념 요약

| 개념 | 설명 | 중요도 |
|------|------|-------|
| **SM** | GPU의 병렬 처리 단위, H100은 132개 SM | ⭐⭐⭐ |
| **Warp** | 32개 Thread가 SIMD 방식으로 동시 실행 | ⭐⭐⭐ |
| **Occupancy** | SM의 활성 Warp 비율 → 높을수록 지연 숨기기 좋음 | ⭐⭐ |
| **AI** | FLOPs/Bytes → Compute vs Memory bound 판별 | ⭐⭐⭐ |
| **Roofline** | 실제 성능 상한 = min(Peak FLOPs, AI × Peak BW) | ⭐⭐⭐ |
| **Shared Memory** | Block 내 스레드 간 고속 공유, Tiling에 필수 | ⭐⭐⭐ |

### 핵심 수식

$$\text{AI} = \frac{\text{FLOPs}}{\text{Bytes}}, \quad \text{Roof} = \min\left(P_{compute}, \; \text{AI} \times P_{bw}\right)$$

$$\text{AI}_{ridge}^{H100} = \frac{989 \text{ TFLOPS}}{3.35 \text{ TB/s}} \approx 295 \text{ FLOPs/Byte}$$

### 다음 챕터 예고
**Chapter 11-02: CUDA C++ 확장** — Python 수준의 TensorFlow/PyTorch 연산으로는 표현이 불가능한 **커스텀 CUDA C++ 커널**을 pybind11로 파이썬에 연결하는 방법을 학습한다. 간단한 벡터 연산부터 Shared Memory를 활용하는 행렬 Tiling까지 단계별 구현한다.